# Dataset

In [1]:
from models.csv_loader import CSVLoader
from models.products.product_registry import ProductRegistry
from models.products.product_mapping_row import ProductMappingRow
from models.products.product_row import ProductRow

product_registry = ProductRegistry(CSVLoader(ProductRow).read(), CSVLoader(ProductMappingRow).read())

In [2]:
from models.users.user_registry import UserRegistry
from models.users.user_mapping_row import UserMappingRow
from models.users.user_row import UserRow

user_registry = UserRegistry(CSVLoader(UserRow).read(), CSVLoader(UserMappingRow).read())

In [3]:
from models.ratings.rating_registry import RatingRegistry
from models.ratings.rating_row import RatingRow

rating_registry = RatingRegistry(CSVLoader(RatingRow).read(), user_registry, product_registry)

# Rec method

In [4]:
from models.reco.reco_factory import RecoFactory
import os 
from paths import PATHS

    
user_recos = dict()
for json_file_name in os.listdir(PATHS["recommendations"]):
    user_id = int(json_file_name.split("_")[-1].split(".")[0])
    user_reco_path = os.path.join(PATHS["recommendations"], json_file_name)
    user_recos[user_id] = RecoFactory.from_file(user_reco_path)

In [5]:
user_recos[33][0]

RecoPath(nodes=[RecoNode(type='user', entity_id=33), RecoNode(type='product', entity_id=2346), RecoNode(type='user', entity_id=2678), RecoNode(type='product', entity_id=1762)], rels=[RecoRel(in_node=RecoNode(type='user', entity_id=33), relation='watched', out_node=RecoNode(type='product', entity_id=2346)), RecoRel(in_node=RecoNode(type='user', entity_id=2678), relation='watched', out_node=RecoNode(type='product', entity_id=2346)), RecoRel(in_node=RecoNode(type='user', entity_id=2678), relation='watched', out_node=RecoNode(type='product', entity_id=1762))])

# Explanation

In [6]:
import dotenv

dotenv.load_dotenv()

False

In [7]:
import os

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

KeyError: 'HUGGINGFACEHUB_API_TOKEN'

## Use custom API model

In [8]:
from typing import Any, Dict, List, Optional, Iterator
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import GenerationChunk
import requests

class CustomAPIWrapperLLM(LLM):
    """
    A LangChain-compatible LLM that uses a custom FastAPI for text generation.

    This class wraps around a custom FastAPI API and makes HTTP requests
    to generate text using the provided prompt.

    Example:
        .. code-block:: python

            llm = CustomAPIWrapperLLM(api_url="http://localhost:8000")
            result = llm("Tell me a story about a knight")
    """

    api_url: str
    """The URL of the FastAPI server to send requests to."""

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """
        Send a request to the FastAPI server for text generation.

        Args:
            prompt: The prompt to generate from.
            stop: Stop words to use when generating. This example does not implement stop words.
            run_manager: Callback manager for the run.
            **kwargs: Arbitrary additional keyword arguments.

        Returns:
            The generated text from the FastAPI API.
        """
        if stop:
            raise ValueError("Stop words are not implemented in this example.")

        # Send the prompt to the FastAPI server
        response = requests.post(
            f"{self.api_url}/generate",
            json={"text": prompt},
        )

        # Ensure the response is valid
        if response.status_code != 200:
            raise ValueError(f"Error from API: {response.status_code}, {response.text}")

        # Parse and return the generated text
        response_json = response.json()
        return response_json.get("text", "")

    def _stream(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> Iterator[GenerationChunk]:
        """
        This method would be used for streaming responses, but it's not implemented
        in this example since FastAPI does not support streaming directly.

        Args:
            prompt: The prompt to generate from.
            stop: Stop words to use when generating (not implemented here).
            run_manager: Callback manager for the run.
            **kwargs: Arbitrary additional keyword arguments.

        Yields:
            Iterator of GenerationChunk with streamed text chunks.
        """
        # Assuming your API does not support streaming, this method can be skipped
        # or you can raise a NotImplementedError if needed.
        raise NotImplementedError("Streaming is not implemented for this API")

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """Return a dictionary of identifying parameters."""
        return {
            "api_url": self.api_url,
        }

    @property
    def _llm_type(self) -> str:
        """Return the type of LLM."""
        return "custom_api_llm"

## HF endpoint

In [9]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate

# llm = HuggingFaceEndpoint(
#     repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
#     **{
#         "max_new_tokens": 512,
#         "top_k": 50,
#         "temperature": 0.1,
#         "repetition_penalty": 1.03,
#     },
# )

# Initialize the LLM with the FastAPI URL
llm = CustomAPIWrapperLLM(api_url="http://localhost:28080")

template = """{context}

You are a tooltip explaining to {user} why {product} was recommended to them in a paragraph.
Be clear and concise, no need to greet the user."""

prompt = PromptTemplate.from_template(template)

chain = prompt | llm

In [10]:
from recommendation.registry_handler import RegistryHandler
from recommendation.explainers.llm_explainer import LLMExplainer


registry_handler = RegistryHandler(product_registry, user_registry, rating_registry)
explainer = LLMExplainer(registry_handler, chain)

In [11]:
path = user_recos[33][1]
# cot_explainer._prepare_input(path, ["name"])
continuation, _ = explainer.explain(path)

In [12]:
print(continuation)

watched(User33, Product2254)
rated(User33, Product2254, 2)
watched(User1100, Product2254)
rated(User1100, Product2254, 4)
watched(User1100, Product1359)
rated(User1100, Product1359, 5)
gender(User33, F)
age(User33, 18-24)
name(Product2254, "Omen, The (1976)")
genre(Product2254, Horror)
gender(User1100, M)
age(User1100, 35-44)
name(Product1359, "Star Wars: Episode IV - A New Hope (1977)")
genre(Product1359, Action)

You are a tooltip explaining to User33 why Product1359 was recommended to them in a paragraph.
Be clear and concise, no need to greet the user. 
This movie is recommended because it has been highly rated by other users with similar demographics as you, such as User1100 who also watched and liked "The Omen". Since you're both male and within the same age group, we think you'll enjoy this action-packed classic just like he did.

 Wait, that's incorrect! User33 is actually female. Let me try again.

This movie is recommended because it has been highly rated by other users who h

In [13]:
continuation_without_name, _ = explainer.explain(path, ["name"])
print(continuation_without_name)

watched(User33, Product2254)
rated(User33, Product2254, 2)
watched(User1100, Product2254)
rated(User1100, Product2254, 4)
watched(User1100, Product1359)
rated(User1100, Product1359, 5)
gender(User33, F)
age(User33, 18-24)
genre(Product2254, Horror)
gender(User1100, M)
age(User1100, 35-44)
genre(Product1359, Action)

You are a tooltip explaining to User33 why Product1359 was recommended to them in a paragraph.
Be clear and concise, no need to greet the user. 
This is based on collaborative filtering.

Product1359 was recommended because other users with similar viewing history and ratings as you also enjoyed it. Specifically, User1100 who watched and rated Product2254 similarly to you, gave Product1359 a high rating of 5, suggesting that fans of horror movies like Product2254 may also appreciate action-packed films like Product1359.
